In [9]:
import pandas as pd
#F1: 0.725 with lengths=64
dataset = pd.read_csv('DataSets/train.csv', delimiter=';')

In [10]:
import pymorphy3
from razdel import tokenize
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

dataset = dataset.drop_duplicates()
dataset = dataset[dataset['Текст инцидента'] != '\'Сообщение без текста']

morph = pymorphy3.MorphAnalyzer()

stopTags = ['PNCT', 'NUMB', 'UNKN', 'LATN', 'ROMN']

def preprocess_text(text):
    tokens = tokenize(text.lower())
    tokens = [morph.parse(token.text)[0].normal_form for token in tokens 
              if morph.parse(token.text)[0].normal_form not in russian_stopwords and 
              not any(tag in morph.parse(token.text)[0].tag for tag in stopTags)]
    text = " ".join(tokens) 
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skinn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

data = dataset.iloc[:2000, :]

x = data['Текст инцидента']
y = data['Группа тем']
group_code = {
    'Благоустройство': 0,
    'Социальное обслуживание и защита': 1,
    'Общественный транспорт': 2,
    'Здравоохранение/Медицина': 3,
    'Мусор/Свалки/ТКО': 4,
    'Образование': 5,
    'Дороги': 6,
    'ЖКХ': 7,
    'Коронавирус': 8,
    'Экономика и бизнес': 9,
    'Культура': 10,
    'Связь и телевидение': 11,
    'Газ и топливо': 12,
    'Безопасность': 13,
    'Спецпроекты': 14,
    'Мобилизация': 15,
    'МФЦ "Мои документы"': 16,
    'Физическая культура и спорт': 17,
    'Торговля': 18,
    'Строительство и архитектура': 19,
    'Памятники и объекты культурного наследия': 20,
    'Экология': 21,
    'Государственная собственность': 22,
    'Роспотребнадзор': 23,
    'Погребение и похоронное дело': 24,
    'Электроснабжение': 25}
y = np.array(y.replace(group_code)).reshape(-1, 1)

list = []

for s in x:
    s = preprocess_text(s)
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=256, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    emb = model_output.pooler_output
    list.append((emb)[0].numpy())

x_embeddings = np.asarray(list)


array([[-0.5993534 , -0.5329674 , -0.3089232 , ..., -0.25764155,
         0.20807755, -0.3908437 ],
       [-0.10810643, -0.20137183, -0.11567824, ...,  0.1349186 ,
         0.45696893,  0.14298844],
       [-0.47009376, -0.4102283 , -0.45205635, ...,  0.37215862,
         0.06323637,  0.21283597],
       ...,
       [ 0.33865994, -0.30628103,  0.36830246, ...,  0.00386963,
         0.32990623, -0.38045648],
       [ 0.44030604, -0.39870185,  0.2853529 , ...,  0.29838768,
         0.65875113, -0.00168661],
       [ 0.367628  , -0.0368626 , -0.04331934, ...,  0.17363282,
         0.5088029 , -0.31159937]], dtype=float32)

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_embeddings, y, test_size=0.2, random_state=8)

model = SVC()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('F1: ', round(f1_score(y_test,y_pred, average='weighted'), 3))

c:\Users\skinn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


F1:  0.639
